In [1]:
import torch

from gfos.model.gnn import LayoutModel
from gfos.data.dataset import LayoutDataset, Normalizer
from gfos.data.utils import load_layout

In [2]:
LAYOUT_DIR = r"H:\data\gfos\predict-ai-model-runtime\npz_all\npz\layout"
NORMALIZER_PATH = "../../data/normalizer.json"

SOURCE = "xla"
SEARCH = "random"

In [ ]:
normalizer = Normalizer.from_json(NORMALIZER_PATH, SOURCE, SEARCH)

layout_data = load_layout(
    LAYOUT_DIR,
    model_type=SOURCE,
    compile_type=SEARCH,
)

test_dataset = LayoutDataset(
    layout_data["valid"],
    normalizer=normalizer,
)

In [4]:
model = LayoutModel(node_feat_dim=112, node_config_dim=14, config_neighbor_layer="GATConv", dropout=0.2)
state_dict = torch.load(r"G:\projects\gfos\logs\pipeline\layout_xla_random_20231020_205139\runs\2023-10-20_20-51-39\wandb\run-20231020_205321-jqf8u2lh\files\719_0.6497.pth")
model.load_state_dict(state_dict["state_dict"])

Disable config edge weight


<All keys matched successfully>

In [5]:
model = model.eval().cuda()

In [7]:
from gfos.metrics import LayoutMetrics

In [ ]:
device = "cuda"
batch_size = 100

results = {}
metric = LayoutMetrics()

with torch.no_grad():
    for record in test_dataset:
        model_id = record["model_id"]
        node_feat = record["node_feat"]
        node_opcode = record["node_opcode"]
        edge_index = record["edge_index"]
        node_config_feat = record["node_config_feat"]
        node_config_ids = record["node_config_ids"]
        config_runtime = record["config_runtime"]
        config_edge_index = record["config_edge_index"]
        config_edge_weight = record["config_edge_weight"]

        (
            node_feat,
            node_opcode,
            edge_index,
            node_config_feat,
            node_config_ids,
            config_edge_index,
            config_edge_weight,
        ) = (
            node_feat.to(device),
            node_opcode.to(device),
            edge_index.to(device),
            node_config_feat.to(device),
            node_config_ids.to(device),
            config_edge_index.to(device),
            config_edge_weight.to(device),
        )

        c = len(config_runtime)
        outs = []

        for i in range(0, c, batch_size):
            end_i = min(i + batch_size, c)
            out: torch.Tensor = model(
                node_feat,
                node_opcode,
                edge_index,
                node_config_feat[i:end_i],
                node_config_ids,
                config_edge_index,
                config_edge_weight,
            )
            outs.append(out.detach().cpu())
        
        outs = torch.concat(outs)
        metric.add(model_id, outs.numpy(), config_runtime.numpy())
        results[model_id] = outs

In [16]:
metric.compute_scores()

{'raw_kendall': 0.6496883592180839,
 'index_kendall': 0.6496372761495242,
 'top100_error': 0.5171428571428571,
 'top500_error': 0.3691428571428572,
 'kendall_bert_pretraining.4x4.fp16': 0.9169770718308861,
 'kendall_inception_v3_batch_128_train': 0.42640980680715157,
 'kendall_mlperf_bert_batch_24_2x2': 0.603722429376769,
 'kendall_resnet50.4x4.fp16': 0.9342399696623834,
 'kendall_resnet_v1_50_official_batch_128_bf16': 0.3862520869562833,
 'kendall_tf2_bert_pretrain_dynamic_batch_size': 0.8202788338324611,
 'kendall_unet_3d.4x4.bf16': 0.4595807345807346}

In [15]:
import pickle


pickle.dump(results, open("results.pkl", "wb"))